# Recognition in natural environments II : Image Data Generation

### Librairies

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_custom_objects
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Suppress deprecation warnings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

### Loading images

Data are loaded by batch during training time and prediction time to avoid crash kernel issue. It's the reason why we are using a start and limit parameters.

In [ ]:
def load_images(image_dir, labels_dir, start, limit):
    # Initialization of variables
    images = []    # List to store input images
    labels = []    # List to store labels images
    filenames = [] # List to store file names

    # List all jpg files and sort them alphabetically
    all_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.jpg')])

    # Slice the sorted list to get the required range
    selected_files = all_files[start:start+limit]

    for filename in selected_files:
        # Reading images
        image = cv2.imread(os.path.join(image_dir, filename))
        label = cv2.imread(os.path.join(labels_dir, filename.replace('.jpg', '.png')), cv2.IMREAD_GRAYSCALE)

        if image is not None and label is not None:
            images.append(cv2.resize(image, (428, 240)))  # Resize to 428x240
            labels.append(cv2.resize(label, (428, 240)))  # Resize to 428x240
            filenames.append(filename)

    return np.array(images), np.array(labels), filenames

### Pre-processing : Normalization of inputs and binarisation of labels

In [ ]:
# Preprocess images and labels
def preprocess_data(images, labels):
    # Inputs normalization
    images = images.astype('float32') / 255.0

    # Labels binarization
    labels = (labels > 0).astype('float32')
    labels = np.expand_dims(labels, axis=-1)

    return images, labels

## Predictions model

### UNet
This model give the best predictions (including post-processing)

In [ ]:
# U-Net model
def UNet(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Downsample
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D((2, 2))(conv1)

    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D((2, 2))(conv2)

    # Bottleneck
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)

    # Upsample
    u1 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv3)
    u1 = layers.concatenate([u1, conv2])
    conv4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    conv4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv4)

    u2 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv4)
    u2 = layers.concatenate([u2, conv1])
    conv5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    conv5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv5)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(conv5)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    
    return model

### UNet more complex
Don't works because the kernel of the laptop crash

In [ ]:
def UNet_deeper(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Downsample
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D((2, 2))(conv1)

    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D((2, 2))(conv2)

    # Bottleneck
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)

    # Upsample
    u1 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv3)
    u1 = layers.concatenate([u1, conv2])
    conv4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    conv4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv4)

    u2 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv4)
    u2 = layers.concatenate([u2, conv1])
    conv5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    conv5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv5)

    # Additional layers
    conv6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv5)
    conv6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)
    pool3 = layers.MaxPooling2D((2, 2))(conv6)

    conv7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool3)
    conv7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
    pool4 = layers.MaxPooling2D((2, 2))(conv7)

    # Upsample
    u3 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(pool4)
    u3 = layers.concatenate([u3, conv7])
    conv8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u3)
    conv8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)

    u4 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    u4 = layers.concatenate([u4, conv6])
    conv9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u4)
    conv9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    
    return model


### VNet
Tested, but less interesting than UNet

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, ZeroPadding2D

def conv_block(inputs, num_filters, kernel_size=(3, 3), activation='relu', padding='same'):
    x = Conv2D(num_filters, kernel_size, padding=padding)(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Conv2D(num_filters, kernel_size, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    return x

def VNet(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    c1 = conv_block(inputs, 16)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    c2 = conv_block(p1, 32)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    c3 = conv_block(p2, 64)
    p3 = layers.MaxPooling2D((2, 2))(c3)
    c4 = conv_block(p3, 128)

    # Decoder
    u1 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c4)
    # Adjust dimensions if necessary
    if c3.shape[1] != u1.shape[1] or c3.shape[2] != u1.shape[2]:
        u1 = ZeroPadding2D(((0, c3.shape[1] - u1.shape[1]), (0, c3.shape[2] - u1.shape[2])))(u1)
    u1 = layers.concatenate([u1, c3])
    c5 = conv_block(u1, 64)

    u2 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c5)
    # Adjust dimensions if necessary
    if c2.shape[1] != u2.shape[1] or c2.shape[2] != u2.shape[2]:
        u2 = ZeroPadding2D(((0, c2.shape[1] - u2.shape[1]), (0, c2.shape[2] - u2.shape[2])))(u2)
    u2 = layers.concatenate([u2, c2])
    c6 = conv_block(u2, 32)

    u3 = layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c6)
    # Adjust dimensions if necessary
    if c1.shape[1] != u3.shape[1] or c1.shape[2] != u3.shape[2]:
        u3 = ZeroPadding2D(((0, c1.shape[1] - u3.shape[1]), (0, c1.shape[2] - u3.shape[2])))(u3)
    u3 = layers.concatenate([u3, c1])
    c7 = conv_block(u3, 16)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c7)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    
    return model

### Loss function for neural networks

In [ ]:
# Custom weighted binary cross-entropy loss
def weighted_binary_crossentropy(y_true, y_pred):
    # Initialization of variables
    weight = 34                           # Value from optimal_weight (represent imbalance in the dataset)
    epsilon = tf.keras.backend.epsilon()  # Small value to avoid numerical instability
    
    # Clip predicted values to avoid log(0) or log(1)
    y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)

    # Calculate the loss for each pixel
    loss = -(weight * y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred))
    
    # Compute the mean loss over all pixels
    return tf.reduce_mean(loss)

Method used once to calculate and initialize the optimal weight of weighted_binary_crossentropy due to the imbalance of weights (lot of black pixels)

In [ ]:
# Calculate the optimal weight based on the imbalance in the dataset
def optimal_weight(boundaries):
    white_pixel = 0
    black_pixel = 0
    # Iterate through each image
    for image in boundaries:
        # Count white and black pixels
        for i in range(240):
            for j in range(428):
                if image[i][j][0] == 0:  # Assuming the image is in RGB format and black pixels have R=0
                    black_pixel += 1
                else:
                    white_pixel += 1
    # Calculate the proportion of white pixels
    white_proportion = white_pixel / (white_pixel + black_pixel)
    # Calculate the optimal weight based on the proportion of white pixels
    optimal_weight = (1 - white_proportion) / white_proportion
    print("Number of white pixels:", white_pixel)
    print("Number of black pixels:", black_pixel)
    print("The optimal weight is:", optimal_weight)

### Plotting function

In [ ]:
# Plot sample
def plot_sample(X, y, preds, ix=None):
    if ix is None:
        ix = np.random.randint(0, len(X))
    has_mask = y[ix].max() > 0

    fig, ax = plt.subplots(1, 3, figsize=(20, 10))
    ax[0].imshow(X[ix])
    if has_mask:
        ax[0].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[0].set_title('Original Image')

    ax[1].imshow(y[ix].squeeze(), cmap='gray')
    ax[1].set_title('True Mask')

    ax[2].imshow(preds[ix].squeeze(), vmin=0, vmax=1, cmap='gray')
    if has_mask:
        ax[2].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[2].set_title('Predicted Mask')
    
    plt.show()

### Save predictions

Mainly used to apply the post-processing

In [ ]:
# Save predictions
def save_predictions(predictions, filenames, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for pred, filename in zip(predictions, filenames):
        pred_image = (pred.squeeze() * 255).astype(np.uint8)
        save_path = os.path.join(save_dir, filename.replace('.jpg', '_pred.png'))
        cv2.imwrite(save_path, pred_image)

### Train model

In [ ]:
# Initialization of variables
image_dir = "./data/train/img/"           # Inputs path directory
boundary_dir = "./data/train/label_img/"  # Labels path directory

# Parameters for the training time
total_images = len(os.listdir(image_dir))  # Number of inputs/data
batch_data = 500                           # Used to cut the training time (to avoid kernel crash : model trained for cut of 500 images)
epochs = 15                                # Number of epochs for the training time
batch_size = 5                             # Number of batch size for the training time
input_shape = (240, 428, 3)                # Adjusted to 428x240 image size
model_save = "unet_model"

# Create and compile the model
model = UNet(input_shape)
model.compile(optimizer='adam', loss=weighted_binary_crossentropy, metrics=['accuracy'])
model.summary()

In [ ]:
# Train the model in batches
for start in range(0, total_images, batch_data):
    print(f"Loading images {start} to {start+batch_data}")

    # Loading inputs and labels
    images, labels, filenames = load_images(image_dir, boundary_dir, start, batch_data)

    # If any data remains
    if len(images) == 0 or len(labels) == 0:
        continue

    # Preprocessing on data
    images, labels = preprocess_data(images, labels)

    # Split data in train and validation dataset
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

    # Save the model after each batch
    model.save(f"{model_save}.h5")

### Prediction on validation dataset to test and visualize first results

In [ ]:
# Predict on validation set
valid_preds = model.predict(X_val, verbose=1)
valid_preds = (valid_preds > 0.5).astype(np.float32)

# Plot some examples
plot_sample(X_val, y_val, valid_preds)

## Predict the model
The model predict 500 images by 500 images to avoid the crash of the laptop

In [ ]:
get_custom_objects().update({"weighted_binary_crossentropy": weighted_binary_crossentropy})

# Load the model with the custom loss function
model_path = f"{model_save}.h5"
model = load_model(model_path, custom_objects={"weighted_binary_crossentropy": weighted_binary_crossentropy})

# Initialization of variables
image_to_predict_dir = "./data/train/img/"            # Images to predict path directory
save_dir = "predictions/first_pred"                   # Save prediction path directory
total_images = len(os.listdir(image_to_predict_dir))  # Number of images to predict
start_index = 0                                       # Start index used to start to load data at this point
num_images_to_process = total_images                  # Number of data to load (if the kernel crash)
batch_data = 500                                      # Used to cut the training time (to avoid kernel crash : model trained for cut of 500 images)

# Load and preprocess the data
images, boundaries, filenames = load_images(image_to_predict_dir, boundary_dir, start_index, num_images_to_process)
images, boundaries = preprocess_data(images, boundaries)

for i in range(0, num_images_to_process, batch_data):
    end_index = min(i + batch_data, num_images_to_process)
    
    # Select the batch of images and boundaries
    batch_images = images[i:end_index]
    batch_filenames = filenames[i:end_index]

    # Prediction on batch of images
    preds = model.predict(batch_images, verbose=1)
    preds = (preds > 0.5).astype(np.float32)

    # Saving predictions and displaying a message
    save_predictions(preds, batch_filenames, save_dir)
    print(f"Images from {i} to {end_index} saved")

## Post-processing

After predictions, we used some post-processing technics to improve results.

Best method : Reducing noise by morpholocigal technics and keep the largest shape detected on the image

In [ ]:
# Function to remove noise by applying morphological opening and closing
def remove_noise_opening_closing(image, opening_kernel_size=(5, 5), closing_kernel_size=(5, 5)):
    # Define kernels for opening and closing operations
    kernel_open = np.ones(opening_kernel_size, np.uint8)
    kernel_close = np.ones(closing_kernel_size, np.uint8)
    
    # Apply morphological opening
    opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel_open)
    
    # Apply morphological closing
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel_close)
    
    return closing

# Function to extract largest connected component from binary image
def largest_connected_component(image, opening_kernel_size=(5, 5)):
    # Threshold the image to binary
    _, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    
    # Define kernel for opening operation
    kernel = np.ones(opening_kernel_size, np.uint8)
    
    # Apply morphological opening
    opened_image = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel)
    
    # Find connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(opened_image, connectivity=8)
    
    # Find index of largest component (excluding background)
    largest_component = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
    
    # Create output image with only the largest component
    output_image = np.zeros_like(opened_image)
    output_image[labels == largest_component] = 255
    
    return output_image

# Function for post-processing predicted images
def post_processing(input_dir, output_dir):
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Get list of predicted image filenames
    predicted_images = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

    # Iterate over each predicted image
    for pred_image_name in predicted_images:
        # Load predicted image
        pred_image_path = os.path.join(input_dir, pred_image_name)
        pred_image = cv2.imread(pred_image_path, cv2.IMREAD_GRAYSCALE)

        # Define opening and closing kernel sizes
        open_size=(5,5)
        close_size=(5,5)

        # Remove noise and extract largest component
        denoised_image = remove_noise_opening_closing(pred_image, open_size, close_size)
        largest_component_image = largest_connected_component(denoised_image, open_size)

        # Save post-processed image
        post_processed_image_path = os.path.join(output_dir, pred_image_name)
        cv2.imwrite(post_processed_image_path, largest_component_image)

# Initialization of variables
input_dir = "predictions/first_pred"       # Images to post-process (in this case : predictions by UNet)
output_dir = "predictions/post_processed"  # Images post-processed

# Post-processing
post_processing(input_dir, output_dir)

### Edge reduction 
With this method, we wanted to reduce the edge size by using a laplacian filter to remove edges from the predictions using substraction. 

In [ ]:
# Edge detection by using Laplacian filter or Canny method
def edge_detection(image, scale=0.5, method="laplacian"):
    if method == "laplacian":
        laplacian = cv2.Laplacian(image, cv2.CV_64F)
        laplacian = np.uint8(np.absolute(laplacian) * scale)
        return laplacian
    elif method == "canny":
        # Ensure binary image using Otsu's thresholding
        _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Apply Canny edge detection
        edges = cv2.Canny(binary_image, 100, 200) # Adjust thresholds if needed

        return edges

# Function to apply 3 edge detection to reduce the edge of the prediction
# Substract the edge of each step and return the prediction with 3 fewer edges
def remove_3edges(prediction):
    # Binarization
    binary_image = (prediction > 127).astype(np.uint8) * 255

    # Apply Edge detection the first time (default : Laplacian filter)
    edges1 = edge_detection(binary_image)
    subtracted1 = cv2.subtract(binary_image, edges1)

    # Apply Edge detection the second time (default : Laplacian filter)
    edges2 = edge_detection(subtracted1)
    subtracted2 = cv2.subtract(subtracted1, edges2)

    # Apply Edge detection the third time (default : Laplacian filter)
    edges3 = edge_detection(subtracted2)
    subtracted3 = cv2.subtract(subtracted2, edges3)

    return subtracted3

# Function to apply X edge detection to reduce the edge of the prediction
# Substract the edge of each step and return the prediction with X fewer edges
# While removing edge it's possible, continue to remove edge
def remove_some_edges(prediction, threshold=1000, max_iter=5, scale=0.5):
    # Binarization
    binary_image = (prediction > 127).astype(np.uint8) * 255
    
    # Initialization of variables
    previous_image = binary_image  # Previous image
    i = 0                          # Loop counter        
    
    # While edge can be removed 
    while i < max_iter:
        edge = edge_detection(previous_image, scale)
        subtracted_image = cv2.subtract(previous_image, edge)

        # Difference between the current and previous image
        difference = np.sum(np.abs(subtracted_image - previous_image))

        # If the difference is below the threshold, stop the loop
        if difference < threshold:
            break

        previous_image = subtracted_image
        i += 1

    return previous_image

def subtract_images(input_dir, output_dir, threshold=1000, max_iterations=5, scale=0.5):
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Process each image in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(".png") or filename.endswith(".jpg"):
            image_path = os.path.join(input_dir, filename)
            prediction = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            if prediction is not None:
                result = remove_some_edges(prediction, threshold, max_iterations, scale)
                # result = remove_3edges(prediction)
                output_path = os.path.join(output_dir, filename)
                cv2.imwrite(output_path, result)

# Initialization of variables
input_dir = "predictions/post_processed"      # Images post-processed
output_dir = "predictions/subtracted_images"  # Substraction between post-processed and edges detection images  

# Perform the image subtraction
subtract_images(input_dir, output_dir)

## Line-detection
It's very difficult to get good results: we need to adjust perfectly the HoughLinesP parameters and have straight lines to be effective.

In [ ]:
def detect_lines(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # Use Canny edge detector
    edges = cv2.Canny(blurred, 50, 150, apertureSize=3)
    # Apply Hough Line Transform
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=10, minLineLength=50, maxLineGap=10)

    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 2)

    return line_image

def line_detection(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith(".png") or filename.endswith(".jpg"):
            image_path = os.path.join(input_dir, filename)
            image = cv2.imread(image_path)
            if image is not None:
                result = detect_lines(image)
                output_path = os.path.join(output_dir, filename)
                cv2.imwrite(output_path, result)

# Initialization of variables
input_dir = "predictions/post_processed"  # Images post-processed
output_dir = "predictions/detect_lines"   # Images with lines detected

# Process images
line_detection(input_dir, output_dir)

## Second predictions on the model

To improve results, we tried to make prediction on predicted data by UNet after a post-processing.

### Load and pre-processing predictions

In [ ]:
def load_pred(image_dir, start, limit):
    # Initialization of variables
    images = []    # List to store input images
    filenames = [] # List to store file names

    # List all jpg files and sort them alphabetically
    all_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.png')])

    # Slice the sorted list to get the required range
    selected_files = all_files[start:start+limit]

    for filename in selected_files:
        # Reading images
        image = cv2.imread(os.path.join(image_dir, filename))

        if image is not None:
            images.append(cv2.resize(image, (428, 240)))  # Resize to 428x240
            filenames.append(filename)

    return np.array(images), filenames

# Preprocess predictions
def preprocess_pred(images):
    # Inputs normalization
    images = images.astype('float32') / 255.0

    return images

### Prediction with VNet

Second prediction was better with VNet pre-trained model than UNet pre-trained model

In [ ]:
get_custom_objects().update({"weighted_binary_crossentropy": weighted_binary_crossentropy})

# Load the model with the custom loss function
model_path = 'vnet_model_4320images.h5'
model = load_model(model_path, custom_objects={"weighted_binary_crossentropy": weighted_binary_crossentropy})

# Initialization of variables
pred_dir = "./predictions_unet_15epochs_5batch_4320images/first_pred"  # Predictions to re-predict path directory
save_dir = "predictions/second_pred"                                   # Save prediction path directory
total_images = len(os.listdir(pred_dir))                               # Number of images to predict
start_index = 0                                                        # Start index used to start to load data at this point
num_images_to_process = total_images                                   # Number of data to load (if the kernel crash)
batch_data = 500                                                       # Used to cut the training time (to avoid kernel crash : model trained for cut of 500 images)

# Load and preprocess the data
images, filenames = load_pred(pred_dir, start_index, num_images_to_process)
images = preprocess_pred(images)

for i in range(0, num_images_to_process, batch_size):
    end_index = min(i + batch_size, num_images_to_process)
    
    # Select the batch of images and boundaries
    batch_images = images[i:end_index]
    batch_filenames = filenames[i:end_index]

    # Prediction on batch of images
    second_preds = model.predict(batch_images, verbose=1)
    second_preds = (second_preds > 0.5).astype(np.float32)

    # Saving predictions and displaying a message
    save_predictions(second_preds, batch_filenames, save_dir)
    print(f"Images from {i} to {end_index} saved")